In [1]:
library("dndscv")
library("readxl")

mutations = read.table('../analysis/merged_maf.txt', 
                       sep="\t", check.names=FALSE, header=TRUE, row.names=NULL)
metasamples = as.data.frame(read_excel('../Metadata.xlsx'))

Warning message:
“replacing previous import ‘Biostrings::translate’ by ‘seqinr::translate’ when loading ‘dndscv’”


In [2]:
head(metasamples)

,PATIENT_ID,SAMPLE_ID,SAMPLE_TYPE,GENDER,INCLUDED,DAYS_AFTER_TREATMENT,AGE_DX,VAR1,VAR2,VAR3,⋯,VAR5,VAR6,VAR7,LANE,FASTA1,FASTA2,ROUND,MSI_ISH,MSI_SCORE,TUMOR_ONLY
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,VHIO_01,1123_3_cfDNA,cfDNA,XX,Y,224,55,BEFORE,F,PD,⋯,2L,1L_2L,triplete,1,s3://scitron/exomes/1123_1.fastq.gz,s3://scitron/exomes/1123_2.fastq.gz,FIRST,MSS,8.32,No
2,VHIO_01,1261_2_cfDNA,cfDNA,XX,Y,310,55,NA,F,PD,⋯,2L,1L_2L,triplete,1,s3://scitron/exomes/1261_1.fastq.gz,s3://scitron/exomes/1261_2.fastq.gz,FIRST,MSS,13.29,No
3,VHIO_01,B19_0970__VH_17_B_022863_A5__1_tumor,tumor,XX,Y,266,55,AFTER,F,PD,⋯,2L,1L_2L,triplete,1,s3://scitron/exomes/B19_0970_S3_R1_001.fastq.gz,s3://scitron/exomes/B19_0970_S3_R2_001.fastq.gz,FIRST,MSS,1.95,No
4,VHIO_01,B19_0971__17_22863_A1__0_gDNA,gDNA,XX,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,1,s3://scitron/exomes/B19_0971_S4_R1_001.fastq.gz,s3://scitron/exomes/B19_0971_S4_R2_001.fastq.gz,FIRST,NA,Na,Na
5,VHIO_02,1514_5_cfDNA,cfDNA,XY,Y,336,57,NA,F,short SD,⋯,2L,1L_2L,doblete,1,s3://scitron/exomes/1514_1.fastq.gz,s3://scitron/exomes/1514_2.fastq.gz,FOURTH,MSS,7.57,No
6,VHIO_02,916_6_cfDNA,cfDNA,XY,Y,35,57,NA,F,short SD,⋯,2L,1L_2L,doblete,1,s3://scitron/exomes/916_1.fastq.gz,s3://scitron/exomes/916_2.fastq.gz,FOURTH,MSS,6.79,No


In [3]:
# Keep only samples that are tumor-normal pair and labeled as included
df = metasamples[(metasamples$INCLUDED == "Y") 
                 & (metasamples$TUMOR_ONLY == "No")
                 & (metasamples$SAMPLE_TYPE != "gDNA")
                 & (metasamples$VAR1 %in% c('BEFORE', 'AFTER')), 
                 c('PATIENT_ID', 'SAMPLE_ID')]
rownames(metasamples) = metasamples$SAMPLE_ID
meta_filtered = metasamples[df$SAMPLE_ID,]
SAMPLES = paste(df$SAMPLE_ID, df$PATIENT_ID, sep="-")
rownames(meta_filtered) = SAMPLES
mutations_filtered = mutations[mutations$Tumor_Sample_Barcode %in% SAMPLES, 
                               c('Tumor_Sample_Barcode', 'Chromosome', 'Start_Position', 'Reference_Allele', 'Tumor_Seq_Allele2')]

In [4]:
sel = rownames(meta_filtered[meta_filtered$VAR1 == 'BEFORE',])
mut = mutations_filtered[mutations_filtered$Tumor_Sample_Barcode %in% sel,]
dndsout = dndscv(mut, refdb="../dndscv_data/data/RefCDS_human_GRCh38.p12.rda", cv=NULL)

sel_cv = dndsout$sel_cv
signif_genes = sel_cv[sel_cv$pglobal_cv < 0.05, c("gene_name", "pglobal_cv", "qglobal_cv")]
rownames(signif_genes) = NULL
signif_genes$pglobal_cv = round(signif_genes$pglobal_cv, digits=3)
signif_genes$qglobal_cv = round(signif_genes$qglobal_cv, digits=3)
write.table(signif_genes, file='../analysis/analysis_before_after/BEFORE/genes_table_dndscv.txt', 
            sep="\t", row.names=FALSE, col.names=TRUE, quote=FALSE)

[1] Loading the environment...

[2] Annotating the mutations...

Warning message in dndscv(mut, refdb = "../dndscv_data/data/RefCDS_human_GRCh38.p12.rda", :
“Same mutations observed in different sampleIDs. Please verify that these are independent events and remove duplicates otherwise.”
[3] Estimating global rates...

Warning message:
“glm.fit: fitted rates numerically 0 occurred”
Warning message:
“glm.fit: fitted rates numerically 0 occurred”
Warning message:
“glm.fit: fitted rates numerically 0 occurred”
[4] Running dNdSloc...

[5] Running dNdScv...

    Regression model for substitutions: no covariates were used (theta = 7.87e-05).

    Regression model for indels (theta = 0.0516)



In [5]:
sel = rownames(meta_filtered[meta_filtered$VAR1 == 'AFTER',])
mut = mutations_filtered[mutations_filtered$Tumor_Sample_Barcode %in% sel,]
dndsout = dndscv(mut, refdb="../dndscv_data/data/RefCDS_human_GRCh38.p12.rda", cv=NULL)

sel_cv = dndsout$sel_cv
signif_genes = sel_cv[sel_cv$pglobal_cv < 0.05, c("gene_name", "pglobal_cv", "qglobal_cv")]
rownames(signif_genes) = NULL
signif_genes$pglobal_cv = round(signif_genes$pglobal_cv, digits=3)
signif_genes$qglobal_cv = round(signif_genes$qglobal_cv, digits=3)
write.table(signif_genes, file='../analysis/analysis_before_after/AFTER/genes_table_dndscv.txt', 
            sep="\t", row.names=FALSE, col.names=TRUE, quote=FALSE)

[1] Loading the environment...

[2] Annotating the mutations...

Warning message in dndscv(mut, refdb = "../dndscv_data/data/RefCDS_human_GRCh38.p12.rda", :
“Same mutations observed in different sampleIDs. Please verify that these are independent events and remove duplicates otherwise.”
[3] Estimating global rates...

Warning message:
“glm.fit: fitted rates numerically 0 occurred”
Warning message:
“glm.fit: fitted rates numerically 0 occurred”
Warning message:
“glm.fit: fitted rates numerically 0 occurred”
[4] Running dNdSloc...

[5] Running dNdScv...



ERROR: Error in while ((it <- it + 1) < limit && abs(del) > eps) {: valor ausente donde TRUE/FALSE es necesario
